In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pwd
%cd /content/drive/MyDrive/Colab Notebooks/term_project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content
/content/drive/MyDrive/Colab Notebooks/term_project


In [2]:
# 최초 1회만 실행

import tarfile

data_root = 'dataset'
tar = tarfile.open("dataset/cifar-10-python.tar.gz")
tar.extractall(data_root)
tar.close()

In [3]:
import numpy as np 
import matplotlib.pyplot as plt

### Data load

In [4]:
from data_utils import load_CIFAR10

cifar10_dir = 'dataset/cifar-10-batches-py'

X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

### Data preprocessing

In [5]:
# X_train = X_train.reshape(len(X_train.squeeze()), -1)
# X_test = X_test.reshape(len(X_test.squeeze()), -1)

print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


### model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

In [7]:
# Define CNN model class
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(p=0.25)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout2 = nn.Dropout(p=0.25)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.dropout3 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.permute(0, 3, 1, 2) # Change data format from (N,H,W,C) to (N,C,H,W)
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        x = self.conv4(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        x = x.view(-1, 64 * 8 * 8)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout3(x)
        x = self.fc2(x)
        return x

### Train

In [8]:
def train(model, X_train_tensor, y_train_tensor, num_epoch = 20, batch_size = 16):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    for epoch in range(num_epoch):
        running_loss = 0.0
        for i in range(0, len(X_train_tensor), batch_size):
            inputs = X_train_tensor[i:i+batch_size]
            labels = y_train_tensor[i:i+batch_size]

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print training progress
        if epoch == 0 or (epoch+1) % 2 == 0 :
              print("[epoch %d / %d] average loss : %f" % (epoch+1, num_epoch, running_loss/len(X_train_tensor)))

### Model

In [9]:
def model(X) :
    # Convert training data to tensor
    X_train_tensor = torch.Tensor(X_train)
    y_train_tensor = torch.Tensor(y_train).long()

    # Create CNN model instance
    model_CNN = CNN()

    # Train model
    train(model_CNN, X_train_tensor, y_train_tensor)

    # Convert data to tensor
    X_tensor = torch.Tensor(X)

    # Make predictions on test data
    preds = model_CNN(X_tensor)

    # Get predicted labels
    _, predicted_labels = torch.max(preds.data, 1)

    # Convert predicted labels to numpy array
    predicted_labels = predicted_labels.numpy()

    return predicted_labels

### Evaluation

In [10]:
# evaluation on test dataset

preds = model(X_test) # (10000,)

print("accuracy on test dataset : ", np.mean(preds==y_test))

[epoch 1 / 20] average loss : 0.111386
[epoch 2 / 20] average loss : 0.083112
[epoch 4 / 20] average loss : 0.067095
[epoch 6 / 20] average loss : 0.060258
[epoch 8 / 20] average loss : 0.056762
[epoch 10 / 20] average loss : 0.054905
[epoch 12 / 20] average loss : 0.053550
[epoch 14 / 20] average loss : 0.052803
[epoch 16 / 20] average loss : 0.052775
[epoch 18 / 20] average loss : 0.052710
[epoch 20 / 20] average loss : 0.052429
accuracy on test dataset :  0.641
